In [ ]:
# import & logging preliminaries
import logging
import itertools

import gensim
from gensim.parsing.preprocessing import STOPWORDS

logging.basicConfig(format='%(levelname)s : %(message)s', level=logging.INFO)
logging.root.level = logging.INFO

In [1]:
import re 
import string
import unicodedata
import sys

STOPWORDS = [u"ấy", u"bị", u"bởi", u"cả", u"các", u"cái", u"cần", u"càng", u"chỉ", u"chiếc", u"cho", u"chứ", u"chưa", 
             u"chuyện", u"có", u"có_thể", u"cứ", u"của", u"cùng", u"cũng", u"đã", u"đang", u"đây", u"để", u"đến_nỗi", 
             u"đều", u"điều", u"do", u"đó", u"được", u"dưới", u"gì", u"khi", u"không", u"là", u"lại", u"lên", u"lúc", 
             u"mà", u"mỗi", u"một cách", u"này", u"nên", u"nếu", u"ngay", u"nhiều", u"như", u"nhưng", u"những", u"nơi", 
             u"nữa", u"phải", u"qua", u"ra", u"rằng", u"rất", u"rồi", u"sau", u"sẽ", u"so", u"sự", u"tại", u"theo", 
             u"thì", u"trên", u"trước", u"từ", u"từng", u"và", u"vẫn", u"vào", u"vậy", u"vì", u"việc", u"với", u"vừa",
             u"_num", u"wwdateww", u"wwtimeww", u"wwemailww", u"wwipww", u"wwurlww", u"wwnumberww"
            ]

tbl = dict.fromkeys(i for i in xrange(sys.maxunicode)
                      if unicodedata.category(unichr(i)).startswith('P') and i != 45 and i!= 95)

def vi_trans_unicode(su):
    return su.translate(tbl)

def vi_strip_text2(s):
    s = re.sub(r"&amp;", "", s)
    s = vi_trans_unicode(s)
    s = re.sub(r"<([^>]+)>", "", s)
    s = re.sub(r"(\s|\\n|\\r|\\t)+", " ", s)
    s = re.sub(r"__+", "_", s)
    s = re.sub(r"--+", "-", s)
    s = re.sub(r'(.)\1+', r'\1\1', s)
    s = ' '.join([w if not w[0].isdigit() else u"wwNUMBERww" for w in s.strip().split()])
    return s


def vi_clean3(line):
    words = line.replace('.','').strip().split()
    words = [w.lower() for w in words]
    return ' '.join(words)

def vi_remove_stop_1char(line):
    words = line.split()
    words = [w for w in words if w not in STOPWORDS and len(w) > 1]
    return ' '.join(words)

def vi_clean4(line):
    words = line.replace('.','').strip().split()
    words = [w.lower() for w in words if len(w) > 1]
    words = [w for w in words if w not in STOPWORDS]
    return ' '.join(words)

import requests
import codecs
import json

tok_url = "http://192.168.0.215:8081/api/v1.0/document/filter"

def get_tokens(tok_url, data):
    try:
        rq = requests.post(tok_url, data=data.encode('utf-8'))
        if rq.content:
            tok_doc = ' '.join(json.loads(rq.content)['sentences'])
        else:
            tok_doc = None
        return tok_doc
    except Exception, e:
        print e

## Corpus, Dict, Model

In [2]:
from gensim import corpora, models, similarities

WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.


In [3]:
# load compact dict
dictionary = corpora.Dictionary.load("/home/laampt/nlp/train/news/lsi/news_180Kdoc.dict")

In [4]:
tfidf = models.TfidfModel.load("/home/laampt/nlp/train/news/lsi/news_180Kdoc.tfidf.mm")

In [5]:
lsi = models.LsiModel.load("/home/laampt/nlp/train/news/lsi/news_180Kdoc.lsi.mm")

In [ ]:
vc = corpora.MmCorpus("/home/laampt/nlp/train/news/lsi/news_180Kdoc.corpus.mm")

## Index

In [6]:
from gensim.similarities.docsim import Similarity

In [7]:
index_tf = Similarity.load('/home/laampt/nlp/train/news/lsi/news_180Kdoc_tfidf.index')

In [8]:
index_lsi = Similarity.load('/home/laampt/nlp/train/news/lsi/news_180Kdoc_lsi.index')

In [ ]:
vc_tfidf = tfidf[vc]

In [ ]:
print vc.num_terms

In [ ]:
index_tf = Similarity(corpus=vc_tfidf, num_features=vc.num_terms, output_prefix="shard")

In [ ]:
index_tf.save('/home/laampt/nlp/train/news/lsi/news_180Kdoc_tfidf.index')

In [ ]:
index_lsi = Similarity(corpus=lsi[vc], num_features=lsi.num_terms, output_prefix="shard")

In [ ]:
index_lsi.save('/home/laampt/nlp/train/news/lsi/news_180Kdoc_lsi.index')

## Query

In [9]:
# new_doc = u"""tình_hình nhà_mạng vnpt viettel mobifone áp_dụng chính_sách chuyển mạng giữ nguyên số"""
new_doc = u"""viettel sẵn_sàng cuộc_cách_mạng về quản_lý đô_thị"""
new_vec = dictionary.doc2bow(new_doc.split())
new_vec_tf = tfidf[new_vec]
new_vec_ls = lsi[new_vec]

In [10]:
print new_vec
print
print new_vec_tf
print
print new_vec_ls

[(16, 1), (406, 1), (671, 1), (1226, 1), (2692, 1), (3170, 1)]

[(16, 0.01694508608699646), (406, 0.05051383542186314), (671, 0.218383837179351), (1226, 0.37282978346291), (2692, 0.5348908438223483), (3170, 0.7241266836026109)]

[(0, 0.42382371632406585), (1, -0.02744548204686223), (2, 0.11866345961701483), (3, -0.049412318513425917), (4, -0.067561021533466561), (5, 0.2562228936184473), (6, 0.32857374323624855), (7, 0.16347042046273125), (8, -0.18353813349241341), (9, 0.21693866949407556), (10, 0.20601846866645332), (11, -0.014049197696204142), (12, -0.51882004324267161), (13, 0.31531127277168142), (14, 0.17246672315108277), (15, -0.2479721695512565), (16, -0.34024124703815262), (17, 0.066546428662634022), (18, -0.082021666060806853), (19, 0.013062636912554555), (20, 0.031746026750234781), (21, -0.057992492529910247), (22, 0.041020993778686253), (23, 0.14671760740502141), (24, -0.29327205363156844), (25, 0.042586158731552026), (26, 0.088091569037370906), (27, 0.19901756104162877), (28,

In [11]:
import heapq

In [12]:
related_story_tfidf = heapq.nlargest(10, enumerate(index_tf[new_vec_tf]), key=lambda x: x[1])
print related_story_tfidf

[(69986, 0.0064971871), (72960, 0.0064951582), (102701, 0.0064836568), (177757, 0.0064768801), (21907, 0.006464791), (10603, 0.0064607318), (97895, 0.0064600417), (178551, 0.006459889), (28450, 0.0064595519), (21840, 0.0064592771)]


In [13]:
related_story_lsi = heapq.nlargest(10, enumerate(index_lsi[new_vec_ls]), key=lambda x: x[1])
print related_story_lsi

[(69218, 0.76624542), (123074, 0.75749052), (31732, 0.74598801), (129484, 0.74100411), (164404, 0.72544515), (183178, 0.72370785), (60983, 0.71986336), (87517, 0.71871299), (52587, 0.71085179), (113898, 0.70877767)]


In [15]:
index_tf.num_best = 10
related_story_tfidf = index_tf[new_vec_tf]
print related_story_tfidf

[(69986, 0.0064971870742738247), (72960, 0.0064951581880450249), (102701, 0.006483656819909811), (177757, 0.0064768800511956215), (21907, 0.0064647910185158253), (10603, 0.0064607318490743637), (97895, 0.0064600417390465736), (178551, 0.0064598890021443367), (28450, 0.0064595518633723259), (21840, 0.0064592771232128143)]


In [16]:
index_lsi.num_best = 10
related_story_lsi = index_lsi[new_vec_ls]
print related_story_lsi

[(69218, 0.76624542474746704), (123074, 0.75749051570892334), (31732, 0.74598801136016846), (129484, 0.74100410938262939), (164404, 0.72544515132904053), (183178, 0.72370785474777222), (60983, 0.71986335515975952), (87517, 0.71871298551559448), (52587, 0.71085178852081299), (113898, 0.70877766609191895)]


In [ ]:
%timeit index_lsi[new_vec_ls]

In [ ]:
%timeit heapq.nlargest(10, enumerate(index_lsi[new_vec_ls]), key=lambda x: x[1])

In [17]:
print related_story_tfidf
print
print related_story_lsi

[(69986, 0.0064971870742738247), (72960, 0.0064951581880450249), (102701, 0.006483656819909811), (177757, 0.0064768800511956215), (21907, 0.0064647910185158253), (10603, 0.0064607318490743637), (97895, 0.0064600417390465736), (178551, 0.0064598890021443367), (28450, 0.0064595518633723259), (21840, 0.0064592771232128143)]

[(69218, 0.76624542474746704), (123074, 0.75749051570892334), (31732, 0.74598801136016846), (129484, 0.74100410938262939), (164404, 0.72544515132904053), (183178, 0.72370785474777222), (60983, 0.71986335515975952), (87517, 0.71871298551559448), (52587, 0.71085178852081299), (113898, 0.70877766609191895)]


In [18]:
line_idx_tfidf = [d for d,v in related_story_tfidf]
line_idx_lsi = [d for d,v in related_story_lsi]

## Check the lines

In [19]:
import numpy as np

In [22]:
results = []
with codecs.open("/home/laampt/nlp/data/news/tok/vt.261K.casen.txt.10word.nodup.clean.nodup", encoding="utf-8", mode='r') as fin:
    for i, line in enumerate(fin):
        if i in line_idx_lsi:
            results.append((i,line))

In [23]:
print new_doc
print '.' * 80
for i, doc in results:
    print "Doc: {}".format(i)
    print doc
    print

viettel sẵn_sàng cuộc_cách_mạng về quản_lý đô_thị
................................................................................
Doc: 31732
bỏ tập_đoàn toàn_cầu về viettel muốn cống_hiến tổ_quốc ictnews_nhiều người từng_làm công_ty nước_ngoài về làm_việc viettel có_một điểm_chung là_ai khát_vọng phục_vụ tổ_quốc mình sự_thật bản_thân tôi về viettel lương tâm_sự một_người đầu_quân viettel from wordpress


Doc: 52587
phó_tổng giám_đốc viettel tiếp_tục đốn_tim cư_dân_mạng hit chắc ai về newsen cư_dân_mạng bày_tỏ thích_thú ngưỡng_mộ xem_clip chắc ai về phó_tổng giám_đốc viettel thể_hiện hãy sử_dụng trình_duyệt hỗ_trợ html thể xem video tpd


Doc: 60983
nước_mắt cầu_thủ viettel viettel thua pvf bán_kết giải_bóng_đá cúp thái_sơn_nam cầu_thủ_trẻ viettel bưng mặt khóc nức không_chỉ giọt_nước_mắt tiếc_nuối không_được chơi trận_chung_kết còn tiếc_nuối hy_vọng về tồn_tại trung_tâm bóng_đá viettel còn vụ gần công_nhân đào_thoát khỏi bãi_vàng quảng_nam điều_tra toàn_diện về ngược_đãi phu_vàng báo_

## Doc2vec

In [24]:
import numpy as np  
from gensim.models import Doc2Vec

In [25]:
dmm = Doc2Vec.load("/home/laampt/nlp/train/news/d2v/news_261K_dmm_5ns_200features_2minwords_5context_e5_lowercase_casentok_nostopword_20epo_alpha_r100")
print dmm

Doc2Vec(dm/m,d200,n5,w5,mc2,s1e-05,t8)


In [26]:
related_story_d2v = []

pos_terms = []
for w in new_doc.split():
    pos_terms.append(dmm[w])
# wvec = dmm.infer_vector(new_doc.split())
related_story_d2v = dmm.docvecs.most_similar(positive=pos_terms)

In [27]:
from collections import defaultdict

In [29]:
print "[*] Getting seed stories ..."
seeds = []
for s,v in dmm.docvecs.most_similar(positive=pos_terms, topn=20):
    if v > 0.3: seeds.append(s)

seeds = list(set(seeds))

print "NUM SEEDS: {}".format(len(seeds))
print '.' * 80

print "[*] Getting depth stories ..."
story_clusters = defaultdict(list)
try:
    for seed in seeds:
        related_stories = [(s,v) for s,v in dmm.docvecs.most_similar([dmm.docvecs[seed]], topn=100) if (v > 0.8) and not(s in seeds)]
        story_clusters[seed] = related_stories
except Exception, e:
    print e
    
print '.' * 80
for seed in seeds:
    related_stories = story_clusters[seed]
    print seed, len(related_stories)
    for story,v in related_stories:
        print "--{}: {}".format(story,v)
    print

[*] Getting seed stories ...
NUM SEEDS: 11
................................................................................
[*] Getting depth stories ...
................................................................................
SENT_vt_18697 2
--SENT_vt_81475: 0.944157600403
--SENT_vt_56362: 0.928602159023

SENT_vt_141782 1
--SENT_vt_68902: 0.817275881767

SENT_vt_77662 0

SENT_vt_86570 0

SENT_vt_63067 0

SENT_vt_149251 2
--SENT_vt_186162: 0.929334998131
--SENT_vt_92904: 0.889158308506

SENT_vt_173718 0

SENT_vt_784 3
--SENT_vt_17614: 0.911697626114
--SENT_vt_109025: 0.850246250629
--SENT_vt_75791: 0.84854722023

SENT_vt_94990 3
--SENT_vt_2699: 0.958082437515
--SENT_vt_48538: 0.947835087776
--SENT_vt_28809: 0.94175362587

SENT_vt_148738 1
--SENT_vt_1557: 0.948173224926

SENT_vt_174425 1
--SENT_vt_48314: 0.8388428092



In [30]:
line_d2v_idx = [int(s.split('_')[-1]) for s in seeds]

In [31]:
results_d2v = []
with codecs.open("/home/laampt/nlp/data/news/tok/vt.261K.casen.txt.10word.nodup.clean.nodup", encoding="utf-8", mode='r') as fin:
    for i, line in enumerate(fin):
        if i in line_d2v_idx:
            results_d2v.append((i,line))

In [32]:
print new_doc
print '.' * 80
for i, doc in results_d2v:
    print "Doc: {}".format(i)
    print doc
    print

viettel sẵn_sàng cuộc_cách_mạng về quản_lý đô_thị
................................................................................
Doc: 784
you are here home tin_tức vì_sao viettel nhận danh_hiệu anh_hùng lực_lượng vũ_trang vì_sao viettel nhận danh_hiệu anh_hùng lực_lượng vũ_trang trong số doanh_nghiệp việt_nam tập_đoàn viễn_thông quân_đội viettel nằm trong số ít nhận hai danh_hiệu anh_hùng lao_động anh_hùng lực_lượng vũ_trang nhân_dân cuối danh_hiệu anh_hùng lao_động doanh_nghiệp thành_tích kinh_doanh xuất_sắc thúc_đẩy tăng_trưởng kinh_tế đóng_góp hoạt_động phát_triển xã_hội trao tuy nhiên danh_hiệu anh_hùng lực_lượng vũ_trang nhân_dân khá đặc_thù doanh_nghiệp cần_có đóng_góp đặc_biệt quan_trọng trong sự_nghiệp xây_dựng bảo_vệ quốc_phòng an_ninh đất_nước trong đạt chỉ_tiêu kinh_doanh mối quan_tâm chiếm hầu hết thời_gian nguồn_lực doanh_nghiệp việt_nam cơ_hội đóng_góp quốc_phòng an_ninh thành_tích đặc_biệt_là rất_hiếm doanh_nghiệp chuyên_ngành quân_đội nghiên_cứu dự_án đặc_biệt phục_vụ

> Infer vector

In [33]:
wvec = dmm.infer_vector(new_doc.split())
print "[*] Getting seed stories ..."
seeds = []
for s,v in dmm.docvecs.most_similar(positive=[wvec], topn=20):
    if v > 0.3: seeds.append(s)

seeds = list(set(seeds))

print "NUM SEEDS: {}".format(len(seeds))
print '.' * 80

print "[*] Getting depth stories ..."
story_clusters = defaultdict(list)
try:
    for seed in seeds:
        related_stories = [(s,v) for s,v in dmm.docvecs.most_similar([dmm.docvecs[seed]], topn=100) if (v > 0.8) and not(s in seeds)]
        story_clusters[seed] = related_stories
except Exception, e:
    print e
    
print '.' * 80
for seed in seeds:
    related_stories = story_clusters[seed]
    print seed, len(related_stories)
    for story,v in related_stories:
        print "--{}: {}".format(story,v)
    print

[*] Getting seed stories ...
NUM SEEDS: 20
................................................................................
[*] Getting depth stories ...
................................................................................
SENT_vt_32257 95
--SENT_vt_141998: 0.936515629292
--SENT_vt_126943: 0.936110436916
--SENT_vt_72598: 0.927281439304
--SENT_vt_173078: 0.925262212753
--SENT_vt_121423: 0.925189197063
--SENT_vt_48448: 0.923503756523
--SENT_vt_37329: 0.922990441322
--SENT_vt_142030: 0.922401487827
--SENT_vt_4776: 0.922245025635
--SENT_vt_91735: 0.921526968479
--SENT_vt_5666: 0.919505417347
--SENT_vt_100675: 0.918270230293
--SENT_vt_91646: 0.91767001152
--SENT_vt_161640: 0.917492628098
--SENT_vt_38945: 0.915853440762
--SENT_vt_145112: 0.915730118752
--SENT_vt_62178: 0.915159642696
--SENT_vt_147708: 0.9149132967
--SENT_vt_125529: 0.91478073597
--SENT_vt_148780: 0.914336323738
--SENT_vt_8842: 0.914001107216
--SENT_vt_79489: 0.913945436478
--SENT_vt_38660: 0.913894236088
--SENT_v

In [34]:
line_d2v_idx = [int(s.split('_')[-1]) for s in seeds]

In [35]:
results_d2v = []
with codecs.open("/home/laampt/nlp/data/news/tok/vt.261K.casen.txt.10word.nodup.clean.nodup", encoding="utf-8", mode='r') as fin:
    for i, line in enumerate(fin):
        if i in line_d2v_idx:
            results_d2v.append((i,line))

In [36]:
print new_doc
print '.' * 80
for i, doc in results_d2v:
    print "Doc: {}".format(i)
    print doc
    print

viettel sẵn_sàng cuộc_cách_mạng về quản_lý đô_thị
................................................................................
Doc: 1545
quý_khách truy_cập bằng wifi vui_lòng đăng_nhập hoặc vui_lòng chuyển sang truy_cập bằng gprs edge viettel bắt_cóc tống_tiền dọa sát_hại con_tin cơ_quan cảnh_sát điều_tra công_an hà_tĩnh ra_quyết_định khởi_tố vụ_án khởi_tố bị_can bắt_tạm_giam đối_tượng về tội bắt_giữ_người trái pháp_luật


Doc: 4429
tuyển gấp nhân_viên chăm sóc khách_hàng mạng viettel mô tả công_việc tư_vấn giải_đáp thắc_mắc khách_hàng liên_quan đến dịch_vụ viettel cung_cấp tiếp_nhận hỗ_trợ xử_lý sự_cố khiếu_nại khách_hàng gặp_phải trong quá_trình sử_dụng dịch_vụ mạng viettel thời_gian làm_việc ca_chính ca đến ca đến ca đến sáng hôm làm_việc ca luân_phiên nghỉ ngày tuần yêu_cầu tốt_nghiệp trung_cấp trở phân_biệt ngành_nghề yêu_cầu kinh_nghiệm hỗ_trợ đào_tạo kỹ_năng giao_tiếp nghiệp_vụ chuyên_môn tham_gia công_việc giá tiền đồng tháng


Doc: 6516
nhận chế_độ khác làm_việc trong một 

## Common sim

In [ ]:
print line_idx
print
print line_d2v_idx

In [ ]:
from sklearn import metrics

In [ ]:
metrics.jaccard_similarity_score(line_idx, line_d2v_idx)